Dash app running on https://explainspiral-penguinkinetic-3000.codio.io/proxy/8050/


In [21]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Configure OS routines
import os, base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD module imports
from CRUD_Python_Module import AnimalShelter
import CRUD_Python_Module as crudmod  # used to resolve absolute path to code_files

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"

db = AnimalShelter(username=username, password=password)

df = pd.DataFrame.from_records(db.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

_code_dir = os.path.dirname(os.path.abspath(crudmod.__file__))
_logo_filename = "Grazioso Salvare Logo.png"
_logo_path = os.path.join(_code_dir, _logo_filename)

encoded_logo = None
if os.path.exists(_logo_path):
    with open(_logo_path, "rb") as f:
        encoded_logo = base64.b64encode(f.read()).decode("ascii")

# helper: age constraint that works whether weeks field exists or only the string field exists
def age_clause(weeks_min, weeks_max):
    weeks_field = "age_upon_outcome_(weeks)"
    regex = None
    if weeks_min == 26 and weeks_max == 156:
        regex = r"(?i)(?:[6-9]\s*months?|1\s*year|2\s*years?|3\s*years?)"
    elif weeks_min == 20 and weeks_max == 300:
        regex = r"(?i)(?:[5-9]\s*months?|[1-5]\s*years?)"
    return {
        "$or": [
            {weeks_field: {"$gte": weeks_min, "$lte": weeks_max}},
            {"age_upon_outcome": {"$regex": regex}}  # fallback when numeric weeks field isn’t present
        ]
    }

def build_query(filter_name: str):
    if filter_name == "All":
        return {}

    if filter_name == "Water Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Labrador Retriever", "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]},
            "sex_upon_outcome": {"$in": ["Neutered Male", "Spayed Female"]},
            **age_clause(26, 156),
            "outcome_type": "Transfer",
        }

    if filter_name == "Mountain or Wilderness Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "German Shepherd", "German Shepherd Mix",
                "Doberman Pinscher",
                "Golden Retriever", "Golden Retriever Mix",
                "Bloodhound",
                "Rottweiler", "Rottweiler Mix"
            ]},
            "sex_upon_outcome": {"$in": ["Intact Male", "Intact Female"]},
            **age_clause(26, 156),
            "outcome_type": "Transfer",
        }

    if filter_name == "Disaster or Individual Tracking":
        return {
            "animal_type": "Dog",
            "breed": {"$in": [
                "German Shepherd", "German Shepherd Mix",
                "Doberman Pinscher",
                "Rottweiler", "Rottweiler Mix",
                "Belgian Malinois",
                "Australian Cattle Dog"
            ]},
            "sex_upon_outcome": {"$in": ["Intact Male", "Intact Female"]},
            **age_clause(20, 300),
            "outcome_type": "Transfer",
        }

    return {}

app.layout = html.Div([
    html.Div([
        html.Div([
            html.Img(
                src=(f"data:image/png;base64,{encoded_logo}" if encoded_logo else None),
                style={"height": "60px"}
            ) if encoded_logo else html.Div(
                f"Logo not found at: {_logo_path}",
                style={"color": "crimson", "fontStyle": "italic", "fontSize": "12px"}
            )
        ]),
        html.Div([
            html.H1("Grazioso Salvare — CS-340 Dashboard"),
            html.P("Student: Caleb Luna")
        ], style={"marginLeft": "15px"})
    ], style={"display": "flex", "alignItems": "center"}),

    html.Hr(),

    html.Center(html.B(html.H3("Interactive Filters"))),

    html.Div([
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "All", "value": "All"},
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
            ],
            value="All",
            labelStyle={"display": "inline-block", "marginRight": "15px"},
        )
    ]),

    html.Hr(),

    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_size=15,
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px"},
        style_header={"backgroundColor": "#f0f0f0", "fontWeight": "bold"},
    ),

    html.Br(),
    html.Hr(),

    html.Div(className="row",
             style={"display": "flex"},
             children=[
                 html.Div(id="graph-id", className="col s12 m6", style={"flex": "1", "paddingRight": "10px"}),
                 html.Div(id="map-id", className="col s12 m6", style={"flex": "1", "paddingLeft": "10px"}),
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output("datatable-id", "data"),
              [Input("filter-type", "value")])
def update_dashboard(filter_type):
    query = build_query(filter_type)
    docs = list(db.read(query))
    df_local = pd.DataFrame.from_records(docs)
    if "_id" in df_local.columns:
        df_local.drop(columns=["_id"], inplace=True)
    return df_local.to_dict("records") if not df_local.empty else []

@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        fig = px.bar(x=[], y=[], title="No data")
        return [dcc.Graph(figure=fig)]

    dff = pd.DataFrame(viewData)

    if "breed" not in dff.columns or dff.empty:
        fig = px.bar(x=[], y=[], title="No breed data")
        return [dcc.Graph(figure=fig)]

    s = dff["breed"].dropna()
    if s.empty:
        fig = px.bar(x=[], y=[], title="No breed data")
        return [dcc.Graph(figure=fig)]

    top = s.value_counts().head(10).reset_index()
    top.columns = ["breed", "count"]
    fig = px.bar(top, x="breed", y="count", title="Top Breeds (Top 10)")
    fig.update_layout(xaxis_tickangle=-35, margin=dict(l=10, r=10, t=40, b=100))
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    return [{
        "if": {"column_id": i},
        "background_color": "#D2F3FF"
    } for i in (selected_columns or [])]

@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return [html.Div("No data to map.")]

    dff = pd.DataFrame.from_dict(viewData)
    row = 0 if not index else index[0]

    lat_col = "location_lat" if "location_lat" in dff.columns else None
    lon_col = "location_long" if "location_long" in dff.columns else None

    if lat_col and lon_col:
        try:
            lat = float(dff.loc[row, lat_col]); lon = float(dff.loc[row, lon_col])
        except Exception:
            lat, lon = 30.75, -97.48
    else:
        try:
            lat = float(dff.iloc[row, 13]); lon = float(dff.iloc[row, 14])
        except Exception:
            lat, lon = 30.75, -97.48

    breed_val = (dff.loc[row, "breed"]
                 if "breed" in dff.columns else (dff.iloc[row, 4] if len(dff.columns) > 4 else ""))
    name_val = (dff.loc[row, "name"]
                if "name" in dff.columns else (dff.iloc[row, 9] if len(dff.columns) > 9 else ""))

    return [
        dl.Map(style={"width": "100%", "height": "500px"},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(str(breed_val)),
                dl.Popup([html.H4("Animal Name"), html.P(str(name_val))])
            ])
        ])
    ]

app.run_server()


Dash app running on https://explainspiral-penguinkinetic-3000.codio.io/proxy/8050/
